# DenseNet 201 model for cancer detection on the HAM10000 dataset

## import necessary libraries

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder

import os
import json

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load preprocessed dataset (resized 128x128, no segmentation)

In [4]:
X = np.load('/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/X_hair_removal_NO_segmentation_128.npy')
y = np.load('/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/y_hair_removal_NO_segmentation_128.npy')

In [6]:
def normalize_images_imagenet(X):
    """
    Normalize images using ImageNet mean and std
    """
    #Scale to [0,1] has already been done
    X = X/255
    #ImageNet normalization
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    for i in range(3):
        X[:,:,:,i] = (X[:,:,:,i] - mean[i]) / std[i]

    return X

In [7]:
X = normalize_images_imagenet(X)

In [8]:
X.shape

(10010, 128, 128, 3)

In [9]:
y.shape

(10010,)

## Define model directories

In [10]:
frozen_model_dir = "/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/Frozen_model/"
fine_tuned_model_dir = "/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/Fine_tuned_model/"

## Prepare labels with One-Hot encoding

In [11]:
def prepare_labels(labels, model_dir):
        """Convert string labels to one-hot encoding"""
        #create and fit label encoder
        label_encoder = LabelEncoder()
        numeric_labels = label_encoder.fit_transform(labels)

        #save label encoder classes so we can use them later for interpretation
        label_mapping = dict(zip(label_encoder.classes_,
                               range(len(label_encoder.classes_))))
        with open(os.path.join(model_dir, 'label_mapping_128.json'), 'w') as f:
            json.dump(label_mapping, f)

        #one-hot encoding the numeric-encoded classes
        one_hot_labels = tf.keras.utils.to_categorical(numeric_labels)

        #Print mapping for verification
        print("Label mapping:")
        for label, idx in label_mapping.items():
            print(f"{label}: {idx}")

        return one_hot_labels, label_encoder

In [12]:
y_encoded, label_encoder = prepare_labels(y, frozen_model_dir)

Label mapping:
akiec: 0
bcc: 1
bkl: 2
df: 3
mel: 4
nv: 5
vasc: 6


In [13]:
y_encoded[0:10]

array([[0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0.]])

### Handle class imbalance -> weigthed loss

first, compute class weights

In [14]:
#CAREFUL use this function with the training set to prevent data leakage
def calculate_class_weights(y_encoded):
    """
    Calculate class weights from original string labels

    Parameters:
    label_encoder: LabelEncoder object which served to encode the original labels
    original_labels: array of original string labels

    Returns:
    dict: mapping of numerical indices to weights
    """
    #Use label encoder to get numerical labels
    numerical_labels = np.argmax(y_encoded, axis=1)

    #Calculate weights
    weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(numerical_labels),
        y=numerical_labels
    )

    #Create dictionary mapping class indices to weights
    class_weights = dict(zip(range(len(weights)), weights))

    return class_weights

In [15]:
#try the function
calculate_class_weights(y_encoded)

{0: 4.386503067484663,
 1: 2.782101167315175,
 2: 1.3035551504102096,
 3: 12.434782608695652,
 4: 1.2848158131176999,
 5: 0.21333731165149933,
 6: 10.070422535211268}

## We're going to do a two phase training approach:
* Initial training with frozen base model
* Fine-tuning of the last 30 layers

### Create a DenseNet model

In [16]:
def create_densenet_model(num_classes, input_shape=(128, 128, 3)):
    """
    Create a DenseNet201 model with custom top layers for melanoma detection
    """
    #Load the pre-trained DenseNet201 model without top layers
    base_model = DenseNet201(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    #Freeze the base model layers
    base_model.trainable = False

    #Add custom top layers
    x = base_model.output
    x = Flatten()(x)
    x = Dropout(0.25)(x)

    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x) #Good habit apparently, It normalizes the activations of each layer, making their means close to 0 and standard deviations close to 1
    x = Dropout(0.5)(x)

    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    #Output layer
    predictions = Dense(num_classes, activation='softmax')(x)

    #Create the full model
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

## Train model with frozen base DenseNet201

In [17]:
def train_model(X, y_encoded, batch_size=32, epochs=50, model_dir='model_checkpoints'):
    """
    Train the model with class weighting and proper checkpoint saving
    """
    #Create model directory if it doesn't exist
    os.makedirs(model_dir, exist_ok=True)

    #Calculate class weights
    class_weights = calculate_class_weights(y_encoded)
    print("Class weights:", class_weights)

    ##---Data splitting: we want a 75, 20, 5 train/test/validation split
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y_encoded,
        test_size=0.20,
        random_state=42, #for reproductibility
        stratify=y_encoded
    )

      #Then split remaining data into train and validation (val is 5% of total)
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val,
        test_size=0.0625,  #0.05/0.80 to get 5% of total data
        random_state=42,
        stratify=y_train_val
    )

    #initialte DenseNet model
    model = create_densenet_model(num_classes=y_encoded.shape[1])

    #compile with optimizers and loss
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')] #AUC is a very good metric for our problem
    )

    #Define callbacks
    checkpoint_path = os.path.join(model_dir, 'densenet201_ph1_128_no_segmentation.keras')
    callbacks = [
        ModelCheckpoint( #save best model at each iteration, because the tensorflow built-in functionnality doesn't work
            checkpoint_path,
            monitor='val_auc',
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        EarlyStopping( #avoid overfitting
            monitor='val_auc', #here we monitor the AUC
            patience=6,
            verbose=1
        ),
        #Set a learning rate annealer
        ReduceLROnPlateau(monitor='val_auc',
                          patience=3,
                          verbose=1,
                          factor=0.5,
                          min_lr=0.00001)
    ]

    #Train the model
    history = model.fit(
        X_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_test, y_test),
        callbacks=callbacks,
        class_weight=class_weights #WEIGHTED LOSS to address class imbalance !
    )

    #Load the best model
    best_model = load_model(checkpoint_path)

    return best_model, history

In [ ]:
best_model_frozen, history_frozen = train_model(X=X, y_encoded=y_encoded, batch_size = 16, epochs = 50, model_dir="/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/Frozen_model/")

Class weights: {0: 4.386503067484663, 1: 2.782101167315175, 2: 1.3035551504102096, 3: 12.434782608695652, 4: 1.2848158131176999, 5: 0.21333731165149933, 6: 10.070422535211268}
74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.2816 - auc: 0.6575 - loss: 2.0661
Epoch 1: val_auc improved from -inf to 0.86396, saving model to /content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/Frozen_model/densenet201_ph1_128_no_segmentation.keras
470/470 ━━━━━━━━━━━━━━━━━━━━ 114s 159ms/step - accuracy: 0.2817 - auc: 0.6576 - loss: 2.0657 - val_accuracy: 0.5430 - val_auc: 0.8640 - val_loss: 1.3221 - learning_rate: 0.0010
Epoch 2/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.4432 - auc: 0.7970 - loss: 1.4312
Epoch 2: val_auc did not improve from 0.86396
470/470 ━━━━━━━━━━━━━━━━━━━━ 21s 44ms/step - accuracy: 0.4432 - auc: 0.7971 - loss: 1.4311 - val_accuracy: 0.5175 - val_auc: 0.8328 - val_loss: 1.46

## Fine tune the last 2 blocks of the model

In [ ]:
def fine_tune_model(model, X, y_encoded, batch_size=128, epochs=30, model_dir='model_checkpoints'):
    """
    Fine-tune the model from phase A with class weighting and proper checkpoint saving
    """
    #Create model directory if it doesn't exist
    os.makedirs(model_dir, exist_ok=True)

    #Calculate class weights
    class_weights = calculate_class_weights(y_encoded)

    ##---Data splitting: we want a 75, 20, 5 train/test/validation split
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y_encoded,
        test_size=0.20,
        random_state=42, #for reproductibility
        stratify=y_encoded
    )

      #Then split remaining data into train and validation (val is 5% of total)
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val,
        test_size=0.0625,  #0.05/0.80 to get 5% of total data
        random_state=42,
        stratify=y_train_val
    )

    #Unfreeze all layers
    for layer in model.layers:
        layer.trainable = True

    #Recompile with a lower learning rate
    model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), ##Much lower learning rate for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )

    #Define callbacks
    checkpoint_path = os.path.join(model_dir, 'densenet201_ph2_YES_dullrazor_NO_segmentation.keras')
    callbacks = [
        ModelCheckpoint(
            checkpoint_path,
            monitor='val_auc',
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        EarlyStopping(
            monitor='val_auc',
            patience=4,
            mode="max",
            verbose=1
        ),
        ReduceLROnPlateau(monitor='val_auc',
                          patience=3,
                          verbose=1,
                          factor=0.5,
                          min_lr=0.000001)
    ]

    #Fine-tune the model
    history = model.fit(
        X_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_test, y_test),
        callbacks=callbacks,
        class_weight=class_weights
    )

    #Load the best fine-tuned model
    best_model = load_model(checkpoint_path)

    return best_model, history

## Load trained frozen model

In [ ]:
frozen_model = load_model("/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/Frozen_model/densenet201_ph1_128_no_segmentation.keras")

## Fine tune the frozen model

In [ ]:
fine_tuned_model, history_fined_tune = fine_tune_model(frozen_model, X=X, y_encoded=y_encoded, batch_size = 16, epochs = 100, model_dir=fine_tuned_model_dir)

Epoch 1/100
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step - accuracy: 0.4783 - auc: 0.8440 - loss: 1.8655
Epoch 1: val_auc improved from -inf to 0.89720, saving model to /content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/Fine_tuned_model/densenet201_ph2_YES_dullrazor_NO_segmentation.keras
470/470 ━━━━━━━━━━━━━━━━━━━━ 542s 554ms/step - accuracy: 0.4783 - auc: 0.8440 - loss: 1.8650 - val_accuracy: 0.5667 - val_auc: 0.8972 - val_loss: 1.1885 - learning_rate: 1.0000e-05
Epoch 2/100
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.5551 - auc: 0.8868 - loss: 1.0639
Epoch 2: val_auc improved from 0.89720 to 0.89795, saving model to /content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/Fine_tuned_model/densenet201_ph2_YES_dullrazor_NO_segmentation.keras
470/470 ━━━━━━━━━━━━━━━━━━━━ 67s 142ms/step - accuracy: 0.5551 - auc: 0.8868 - loss: 1.0639 - val_accuracy: 0.5811 - val_auc: 0.8980 - val_loss: 1.1886 - learning_rate: 1.0000e-05
Ep